In [4125]:
import pandas as pd

In [4126]:
share_energy_df = pd.read_csv('CSVS\\01 renewable-share-energy.csv')
consumption_df = pd.read_csv('CSVS\\02 modern-renewable-energy-consumption.csv')
production_df = pd.read_csv('CSVS\\03 modern-renewable-prod.csv')
solar_cap_df = pd.read_csv('CSVS\\13 installed-solar-PV-capacity.csv')

In [4127]:
merged_df = production_df.merge(consumption_df , on = ['Entity', 'Year', 'Code'], how='outer')
merged_df = merged_df.merge(share_energy_df , on = ['Entity', 'Year', 'Code'], how='outer')
merged_df = merged_df.merge(solar_cap_df, on = ['Entity', 'Year', 'Code'], how='outer')

In [4128]:
# Normalización de los datos y creación de las tablas

In [4129]:
# Crear la tabla de países/regiones sin duplicados
countries_df = merged_df[['Entity', 'Code']].drop_duplicates().reset_index(drop=True)
countries_df['country_id'] = countries_df.index + 1
countries_df = countries_df[['country_id', 'Entity', 'Code']]

In [4130]:
# Crear el DataFrame de continentes utilizando diccionario por registro
data_continentes = [
    {'Continent': 'Africa', 'continent_id': 1},
    {'Continent': 'North America', 'continent_id': 2},
    {'Continent': 'South America', 'continent_id': 3},
    {'Continent': 'Oceania', 'continent_id': 4},
    {'Continent': 'Europe', 'continent_id': 5},
    {'Continent': 'Asia', 'continent_id': 6},
    {'Continent': 'Regions', 'continent_id': 7},
    {'Continent': 'Other', 'continent_id': 8}
]

# Crear el DataFrame
continentes_df = pd.DataFrame(data_continentes)
continentes_df = continentes_df[['continent_id', 'Continent']]

# Usamos un csv con los países y su respectivo continente
continentes_paises_df = pd.read_csv('CSVS\\18 Countries by continents.csv')
continentes_paises_df = continentes_paises_df.rename(columns={'Country':'Entity'})

# Asignar el continente al país usando el csv
countries_df = countries_df.merge(continentes_paises_df, on='Entity', how='left')
print(countries_df[countries_df['Continent'].isna()]['Entity'].unique())

# Confirmamos los países africanos 54/54 + ['Reunion', 'Saint Helena', 'Western Sahara']
africa_countries = [
    "Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cabo Verde", "Cameroon",
    "Central African Republic", "Chad", "Comoros", "Democratic Republic of the Congo", "Cote d'Ivoire",
    "Djibouti", "Egypt", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea",
    "Guinea-Bissau", "Equatorial Guinea", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi",
    "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria",
    "Republic of the Congo", "Rwanda", "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone",
    "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia",
    "Zimbabwe", "Reunion", "Saint Helena", "Western Sahara"
]
countries_df.loc[countries_df['Entity'].isin(africa_countries), 'Continent'] = 'Africa'

# Confirmamos los países 45/49 + ["Faeroe Islands", "Gibraltar", "USSR"] 47/49
europe_countries = [
    "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", 
    "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", 
    "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Liechtenstein", "Lithuania", 
    "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", "North Macedonia", "Norway", 
    "Poland", "Portugal", "Romania", "Russia", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", 
    "Sweden", "Switzerland", "Ukraine", "United Kingdom", "Vatican City", "Armenia", "Azerbaijan", 
    "Georgia", "Turkey", "Faeroe Islands", "Gibraltar", "USSR"
]
countries_df.loc[countries_df['Entity'].isin(europe_countries), 'Continent'] = 'Europe'

# Confirmamos los países de Oceania 10/14 + ["American Samoa","Cook Islands","Guam","New Caledonia","Niue","French Polynesia"] 16/14
oceania_countries = [
    "Australia", "Fiji", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "New Zealand",
    "Palau", "Papua New Guinea", "Samoa", "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu", "American Samoa", "Cook Islands", "Guam", "New Caledonia", "Niue", "French Polynesia"
]
countries_df.loc[countries_df['Entity'].isin(oceania_countries), 'Continent'] = 'Oceania' #Falta palau, tuvalu, Marshall islands y micronesia

# Confirmamos los países de Asia 49/49 + ["Macao", "Timor"] 51/49
asia_countries = [
    "Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Cambodia", 
    "China", "Cyprus", "Georgia", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", 
    "Kazakhstan", "Kuwait", "Kyrgyzstan", "Laos", "Lebanon", "Malaysia", "Maldives", "Mongolia", 
    "Myanmar", "Nepal", "North Korea", "Oman", "Pakistan", "Palestine", "Philippines", "Qatar", 
    "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Tajikistan", "Thailand", 
    "Timor-Leste", "Turkey", "Turkmenistan", "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen", "Macao", "Timor"
]
countries_df.loc[countries_df['Entity'].isin(asia_countries), 'Continent'] = 'Asia'

# Confirmamos los países que hacen parte de North America + ["Bermuda", "Greenland", 'Saint Pierre and Miquelon'] (26)
north_america_countries = ["Bermuda", "Greenland", "Saint Pierre and Miquelon"]
countries_df.loc[countries_df['Entity'].isin(north_america_countries), "Continent"] = "North America"

# Confirmamos los países que hacen parte de South America (Incluyendo islas del caribe)
south_america_countries = ["Aruba","British Virgin Islands","Cayman Islands","Guadeloupe","Martinique","Montserrat","Puerto Rico","Turks and Caicos Islands","United States Virgin Islands","Falkland Islands", "French Guiana"]
countries_df.loc[countries_df['Entity'].isin(south_america_countries), "Continent"] = "South America"

# Other categories
other_categories = ["CIS (BP)", "G20 (Ember)", "G7 (Ember)", "High-income countries", "Low-income countries", "Lower-middle-income countries", "Non-OECD (BP)", "OECD (BP)", "OECD (Ember)", "Upper-middle-income countries", "World"]
countries_df.loc[countries_df['Entity'].isin(other_categories), 'Continent'] = 'Other'

# Regions
regions = [
    "Africa", "Africa (BP)", "Africa (Ember)", "Asia", "Asia (Ember)",
    "Asia Pacific (BP)", "Central America (BP)", "Eastern Africa (BP)", "Europe",
    "Europe (BP)", "Europe (Ember)", "European Union (27)",
    "European Union (27) (Ember)", "Latin America and Caribbean (Ember)",
    "Middle Africa (BP)", "Middle East (BP)", "North America",
    "North America (BP)", "North America (Ember)", "Oceania", "Oceania (Ember)",
    "South America", "South and Central America (BP)", "Western Africa (BP)"
]
countries_df.loc[countries_df['Entity'].isin(regions), 'Continent'] = 'Regions'

['Africa' 'Africa (BP)' 'Africa (Ember)' 'American Samoa' 'Aruba' 'Asia'
 'Asia (Ember)' 'Asia Pacific (BP)' 'Bermuda' 'British Virgin Islands'
 'Burkina Faso' 'CIS (BP)' 'Cayman Islands' 'Central America (BP)'
 'Cook Islands' "Cote d'Ivoire" 'Eastern Africa (BP)' 'Eswatini' 'Europe'
 'Europe (BP)' 'Europe (Ember)' 'European Union (27)'
 'European Union (27) (Ember)' 'Faeroe Islands' 'Falkland Islands'
 'French Guiana' 'French Polynesia' 'G20 (Ember)' 'G7 (Ember)' 'Gibraltar'
 'Greenland' 'Guadeloupe' 'Guam' 'High-income countries' 'Kosovo'
 'Latin America and Caribbean (Ember)' 'Low-income countries'
 'Lower-middle-income countries' 'Macao' 'Martinique' 'Middle Africa (BP)'
 'Middle East (BP)' 'Montserrat' 'Myanmar' 'New Caledonia' 'Niue'
 'Non-OECD (BP)' 'North America' 'North America (BP)'
 'North America (Ember)' 'North Macedonia' 'OECD (BP)' 'OECD (Ember)'
 'Oceania' 'Oceania (Ember)' 'Palestine' 'Puerto Rico' 'Reunion'
 'Saint Helena' 'Saint Pierre and Miquelon' 'South America'
 

In [4131]:
# Crear una nueva columna 'id_continente' y asignar el continente
countries_df = countries_df.merge(continentes_df, on = 'Continent', how = 'left')
countries_df = countries_df.drop(columns='Continent')
countries_df = countries_df.reset_index(drop=True)

In [4132]:
# Crear la tabla de años sin duplicados
years_df = merged_df[['Year']].drop_duplicates().reset_index(drop = True).sort_values('Year')
years_df.insert(0, 'year_id', range(1, len(years_df) + 1))

In [4133]:
# Datos comunes y preparación del values_df
foreign_keys = ['country_id', 'year_id']

In [4134]:
# Crear la tabla de consumo conservando las llaves foráneas
consumption_merged_df = consumption_df.merge(countries_df, on=['Entity', 'Code'], how='left')
consumption_merged_df = consumption_merged_df.merge(years_df, on='Year', how='left')

consumption_merged_df['Year'] = consumption_merged_df['Year'].astype(int) #Convierte columna Year a int  
consumption_merged_df['country_id'] = consumption_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

consumption_merged_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']] = consumption_merged_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']].fillna(0)

consumption_merged_df = consumption_merged_df[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] + foreign_keys]

consumption_merged_df.index += 1

In [4135]:
""" ########################################################################################################################################################### PRUEBA CONSUMO ######################################################################################################################"""

""" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)
hola = consumption_merged_df[consumption_merged_df['continent_id'] == 6]
hola_comp = consumption_merged_df[consumption_merged_df['Entity'] == 'Asia']
 
# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año
suma_hydro = hola.groupby('Year')[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']].sum()

# Obtener los valores individuales de generación hidroeléctrica de cada país
hydro_paises = hola_comp[['Year', 'Entity', 'Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']]

# Crear un DataFrame comparativo
comparacion_df = hydro_paises.merge(suma_hydro, on='Year', suffixes=('_Pais', '_Suma'))

# Agregar una columna de diferencia
comparacion_df['Diferencia_Hydro'] = comparacion_df['Hydro Generation - TWh_Pais'] - comparacion_df['Hydro Generation - TWh_Suma']
comparacion_df['Diferencia_Bio'] = comparacion_df['Geo Biomass Other - TWh_Pais'] -  comparacion_df['Geo Biomass Other - TWh_Suma']
comparacion_df['Diferencia_Solar'] = comparacion_df['Solar Generation - TWh_Pais'] - comparacion_df['Solar Generation - TWh_Suma']
comparacion_df['Diferencia_Wind'] =  comparacion_df['Wind Generation - TWh_Pais'] - comparacion_df['Wind Generation - TWh_Suma']

# Mostrar el DataFrame
comparacion_df[['Diferencia_Hydro', 'Diferencia_Bio', 'Diferencia_Solar', 'Diferencia_Wind']].abs().max() """

" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)\nhola = consumption_merged_df[consumption_merged_df['continent_id'] == 6]\nhola_comp = consumption_merged_df[consumption_merged_df['Entity'] == 'Asia']\n\n# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año\nsuma_hydro = hola.groupby('Year')[['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']].sum()\n\n# Obtener los valores individuales de generación hidroeléctrica de cada país\nhydro_paises = hola_comp[['Year', 'Entity', 'Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh']]\n\n# Crear un DataFrame comparativo\nc

In [4136]:
# Crear la tabla de producción conservando las llaves foráneas
production_merged_df = production_df.merge(countries_df, on=['Entity', 'Code'], how='left')
production_merged_df = production_merged_df.merge(years_df, on='Year', how='left')

production_merged_df['Year'] = production_merged_df['Year'].astype(int) #Convierte columna Year a int  
production_merged_df['country_id'] = production_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

production_merged_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']] = production_merged_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']].fillna(0)

production_merged_df = production_merged_df[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)'] + foreign_keys]

production_merged_df.index += 1

In [4137]:
""" ########################################################################################################################################################### PRUEBA PRODUCCIÓN ######################################################################################################################"""

""" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)
hola = production_merged_df[production_merged_df['continent_id'] == 6]
hola_comp = production_merged_df[production_merged_df['Entity'] == 'Asia']
 
# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año
suma_hydro = hola.groupby('Year')[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']].sum()

# Obtener los valores individuales de generación hidroeléctrica de cada país
hydro_paises = hola_comp[['Year', 'Entity', 'Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']]

# Crear un DataFrame comparativo
comparacion_df = hydro_paises.merge(suma_hydro, on='Year', suffixes=('_Pais', '_Suma'))

# Agregar una columna de diferencia
comparacion_df['Diferencia_Hydro'] = comparacion_df['Electricity from hydro (TWh)_Pais'] - comparacion_df['Electricity from hydro (TWh)_Suma']
comparacion_df['Diferencia_Bio'] = comparacion_df['Other renewables including bioenergy (TWh)_Pais'] -  comparacion_df['Other renewables including bioenergy (TWh)_Suma']
comparacion_df['Diferencia_Solar'] = comparacion_df['Electricity from solar (TWh)_Pais'] - comparacion_df['Electricity from solar (TWh)_Suma']
comparacion_df['Diferencia_Wind'] =  comparacion_df['Electricity from wind (TWh)_Pais'] - comparacion_df['Electricity from wind (TWh)_Suma']

# Mostrar el DataFrame
comparacion_df[['Diferencia_Hydro', 'Diferencia_Bio', 'Diferencia_Solar', 'Diferencia_Wind']].abs().max() """

" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)\nhola = production_merged_df[production_merged_df['continent_id'] == 6]\nhola_comp = production_merged_df[production_merged_df['Entity'] == 'Asia']\n\n# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año\nsuma_hydro = hola.groupby('Year')[['Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electricity from solar (TWh)', 'Other renewables including bioenergy (TWh)']].sum()\n\n# Obtener los valores individuales de generación hidroeléctrica de cada país\nhydro_paises = hola_comp[['Year', 'Entity', 'Electricity from wind (TWh)', 'Electricity from hydro (TWh)', 'Electrici

In [4138]:
# Crear la tabla de porcentaje de energía renovable respecto al total conservando las llaves foráneas
renewable_percent_merged_df = share_energy_df.merge(countries_df, on=['Entity', 'Code'], how='left')
renewable_percent_merged_df = renewable_percent_merged_df.merge(years_df, on='Year', how='left')

renewable_percent_merged_df['Year'] = renewable_percent_merged_df['Year'].astype(int) #Convierte columna Year a int  
renewable_percent_merged_df['country_id'] = renewable_percent_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

renewable_percent_merged_df[['Renewables (% equivalent primary energy)']] = renewable_percent_merged_df[['Renewables (% equivalent primary energy)']].fillna(0)

renewable_percent_merged_df = renewable_percent_merged_df[['Renewables (% equivalent primary energy)'] + foreign_keys]
renewable_percent_merged_df.index += 1

In [4139]:
""" ########################################################################################################################################################### PRUEBA PORCENTAJE RENOVABLE ######################################################################################################################"""

""" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)
hola = renewable_percent_merged_df[renewable_percent_merged_df['continent_id'] == 4]
hola_comp = renewable_percent_merged_df[renewable_percent_merged_df['Entity'] == 'Oceania']
 
# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año
suma_hydro = hola.groupby('Year')[['Renewables (% equivalent primary energy)']].mean()

# Obtener los valores individuales de generación hidroeléctrica de cada país
hydro_paises = hola_comp[['Year', 'Entity', 'Renewables (% equivalent primary energy)']]

# Crear un DataFrame comparativo
comparacion_df = hydro_paises.merge(suma_hydro, on='Year', suffixes=('_continent', '_Suma'))

# Agregar una columna de diferencia
comparacion_df['Diferencia_Percent'] = comparacion_df['Renewables (% equivalent primary energy)_continent'] - comparacion_df['Renewables (% equivalent primary energy)_Suma']

# Mostrar el DataFrame
comparacion_df['Diferencia_Percent'].abs().max() """

" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)\nhola = renewable_percent_merged_df[renewable_percent_merged_df['continent_id'] == 4]\nhola_comp = renewable_percent_merged_df[renewable_percent_merged_df['Entity'] == 'Oceania']\n\n# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año\nsuma_hydro = hola.groupby('Year')[['Renewables (% equivalent primary energy)']].mean()\n\n# Obtener los valores individuales de generación hidroeléctrica de cada país\nhydro_paises = hola_comp[['Year', 'Entity', 'Renewables (% equivalent primary energy)']]\n\n# Crear un DataFrame comparativo\ncomparacion_df = hydro_paises.merge(suma_hydro, on='Year', suffixes=('_continent', '_Suma'))\

In [4140]:
# Crear la tabla de capacidad de energía solar conservando las llaves foráneas
solar_capacity_merged_df = solar_cap_df.merge(countries_df, on=['Entity', 'Code'], how='left')
solar_capacity_merged_df = solar_capacity_merged_df.merge(years_df, on='Year', how='left')

solar_capacity_merged_df['Year'] = solar_capacity_merged_df['Year'].astype(int) #Convierte columna Year a int  
solar_capacity_merged_df['country_id'] = solar_capacity_merged_df['country_id'].astype(int) #Convierte columna Country_id a int  

solar_capacity_merged_df[['Solar Capacity']] = solar_capacity_merged_df[['Solar Capacity']].fillna(0)

""" solar_capacity_merged_df = solar_capacity_merged_df[['Solar Capacity'] + foreign_keys]
solar_capacity_merged_df.index += 1 """

" solar_capacity_merged_df = solar_capacity_merged_df[['Solar Capacity'] + foreign_keys]\nsolar_capacity_merged_df.index += 1 "

In [4141]:
""" ########################################################################################################################################################### PRUEBA PORCENTAJE CAPACIDAD SOLAR
###################################################################################################################### """

""" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)
hola = solar_capacity_merged_df[solar_capacity_merged_df['continent_id'] == 6]
hola_comp = solar_capacity_merged_df[solar_capacity_merged_df['Entity'] == 'Asia']
 
# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año
suma_hydro = hola.groupby('Year')[['Solar Capacity']].sum()

# Obtener los valores individuales de generación hidroeléctrica de cada país
hydro_paises = hola_comp[['Year', 'Entity', 'Solar Capacity']]

# Crear un DataFrame comparativo
comparacion_df = hydro_paises.merge(suma_hydro, on='Year', suffixes=('_continent', '_Suma'))

# Agregar una columna de diferencia
comparacion_df['Diferencia_Percent'] = comparacion_df['Solar Capacity_continent'] - comparacion_df['Solar Capacity_Suma']

# Mostrar el DataFrame
comparacion_df['Diferencia_Percent'].abs().max() """

" # Sumar la generación hidroeléctrica de todos los países asíaticos y europeos en ['Geo Biomass Other - TWh', 'Solar Generation - TWh', 'Wind Generation - TWh', 'Hydro Generation - TWh'] (Casi perfecto en Geo, Solar y Wind falla en Hydro por 10 unidades hasta el 75 de ahí en adelante aumenta hasta llegar 2021 con una diferencia de casi 100)\nhola = solar_capacity_merged_df[solar_capacity_merged_df['continent_id'] == 6]\nhola_comp = solar_capacity_merged_df[solar_capacity_merged_df['Entity'] == 'Asia']\n\n# Sumar la generación hidroeléctrica de todos los países asiáticos y europeos por año\nsuma_hydro = hola.groupby('Year')[['Solar Capacity']].sum()\n\n# Obtener los valores individuales de generación hidroeléctrica de cada país\nhydro_paises = hola_comp[['Year', 'Entity', 'Solar Capacity']]\n\n# Crear un DataFrame comparativo\ncomparacion_df = hydro_paises.merge(suma_hydro, on='Year', suffixes=('_continent', '_Suma'))\n\n# Agregar una columna de diferencia\ncomparacion_df['Diferencia_P

In [4142]:
# Exportar los CSVs normalizados
countries_df.to_csv('entities.csv', index=False)
years_df.to_csv('years.csv', index=False)
continentes_df.to_csv('regions.csv',index=False)
consumption_merged_df.to_csv('consumption.csv')
production_merged_df.to_csv('production.csv')
renewable_percent_merged_df.to_csv('renewable_percent.csv')
solar_capacity_merged_df.to_csv('solar_capacity.csv')

In [4143]:
"""
-- 1. Tabla para regions.csv
CREATE TABLE regions (
    region_id INT PRIMARY KEY AUTO_INCREMENT,
    region VARCHAR(255) NOT NULL
);

-- 2. Tabla para entities.csv
CREATE TABLE entities (
    entity_id INT PRIMARY KEY AUTO_INCREMENT,
    entity VARCHAR(255) NOT NULL,
    code VARCHAR(10),
    region_id INT,
    FOREIGN KEY (region_id) REFERENCES regions(region_id) ON DELETE SET NULL ON UPDATE CASCADE
);

-- 3. Tabla para years.csv
CREATE TABLE years (
    year_id INT PRIMARY KEY AUTO_INCREMENT,
    year INT NOT NULL UNIQUE
);

-- 4. Tabla para renewable_energy_consumption.csv
CREATE TABLE renewable_energy_consumption (
    id INT PRIMARY KEY AUTO_INCREMENT,
    bioenergy_TWh FLOAT NOT NULL,
    solar_TWh FLOAT NOT NULL,
    wind_TWh FLOAT NOT NULL,
    hydro_TWh FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 5. Tabla para renewable_energy_production.csv
CREATE TABLE renewable_energy_production (
    id INT PRIMARY KEY AUTO_INCREMENT,
    wind_TWh FLOAT NOT NULL,
    hydro_TWh FLOAT NOT NULL,
    solar_TWh FLOAT NOT NULL,
    bioenergy_TWh FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 6. Tabla para renewable_percent.csv
CREATE TABLE renewable_percent (
    id INT PRIMARY KEY AUTO_INCREMENT,
    renewables_percent FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- 7. Tabla para solar_capacity.csv
CREATE TABLE solar_capacity (
    id INT PRIMARY KEY AUTO_INCREMENT,
    solar_capacity FLOAT NOT NULL,
    entity_id INT NOT NULL,
    year_id INT NOT NULL,
    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (year_id) REFERENCES years(year_id) ON DELETE CASCADE ON UPDATE CASCADE
);

-- Ejemplos Select's
SELECT * FROM entities;
SELECT * FROM years;
SELECT * FROM regions;
SELECT * FROM consumption;
SELECT * FROM production;
SELECT * FROM renewable_percent;
SELECT * FROM solar_capacity;
"""

"\n-- 1. Tabla para regions.csv\nCREATE TABLE regions (\n    region_id INT PRIMARY KEY AUTO_INCREMENT,\n    region VARCHAR(255) NOT NULL\n);\n\n-- 2. Tabla para entities.csv\nCREATE TABLE entities (\n    entity_id INT PRIMARY KEY AUTO_INCREMENT,\n    entity VARCHAR(255) NOT NULL,\n    code VARCHAR(10),\n    region_id INT,\n    FOREIGN KEY (region_id) REFERENCES regions(region_id) ON DELETE SET NULL ON UPDATE CASCADE\n);\n\n-- 3. Tabla para years.csv\nCREATE TABLE years (\n    year_id INT PRIMARY KEY AUTO_INCREMENT,\n    year INT NOT NULL UNIQUE\n);\n\n-- 4. Tabla para renewable_energy_consumption.csv\nCREATE TABLE renewable_energy_consumption (\n    id INT PRIMARY KEY AUTO_INCREMENT,\n    bioenergy_TWh FLOAT NOT NULL,\n    solar_TWh FLOAT NOT NULL,\n    wind_TWh FLOAT NOT NULL,\n    hydro_TWh FLOAT NOT NULL,\n    entity_id INT NOT NULL,\n    year_id INT NOT NULL,\n    FOREIGN KEY (entity_id) REFERENCES entities(entity_id) ON DELETE CASCADE ON UPDATE CASCADE,\n    FOREIGN KEY (year_id) 